In [55]:
import pandas as pd
import numpy as np

import sys
sys.path.append('../../pycore/')
from utils import extract_details_from_rxnid

In [56]:
### Load info
# Enzyme
df_enz = pd.read_excel('../../build_model/input/ENZYME_stoich_curation_2021-10-01.xlsx')
df_enz.index = df_enz.id.to_list()

# Stoichiometry
df_stoich = pd.read_excel('../../build_model/model/RBA_stoichiometry.xlsx')
df_stoich.index = df_stoich.id.to_list()

# Excluded entries
with open('./input/exclude_reactions.txt') as f:
    rxns_excl = f.read().split('\n')[:-1]
    
# kapp
df_kapp = pd.read_excel('./results/kapp_bydatagroup_inPerSec.xlsx')
df_kapp.index = df_kapp.id.to_list()

In [57]:
df_kapp.columns

Index(['id', 'subsystem_group', 'subsystem', 'batch_std', 'batch_rich',
       'batch_anaero', 'chemo_lowC', 'chemo_highC', 'chemo_lowN',
       'chemo_highN', 'batch_Gal', 'batch_Malt', 'batch_Tre'],
      dtype='object')

In [58]:
### Select the growth conditions
cond = 'batch_Tre'
outfile = '../../input/RBA_kapp_batch_Tre_per_hour.txt'

#cond = 'batch_Malt'
#outfile = '../../input/RBA_kapp_batch_Malt_per_hour.txt'

#cond = 'batch_Gal'
#outfile = '../../input/RBA_kapp_batch_Gal_per_hour.txt'

#cond = 'chemo_highN'
#outfile = '../../input/RBA_kapp_chemo_highN_per_hour.txt'

#cond = 'chemo_lowN'
#outfile = '../../input/RBA_kapp_chemo_lowN_per_hour.txt'

#cond = 'chemo_highC'
#outfile = '../../input/RBA_kapp_chemo_highC_per_hour.txt'

#cond = 'chemo_lowC'
#outfile = '../../input/RBA_kapp_chemo_lowC_per_hour.txt'

#cond = 'batch_anaero'
#outfile = '../../input/RBA_kapp_batch_anaero_per_hour.txt'

#cond = 'batch_rich'
#outfile = '../../input/RBA_kapp_batch_rich_per_hour.txt'

#cond = 'batch_std'
#outfile = '../../input/RBA_kapp_batch_per_hour.txt'

In [59]:
kapp0 = df_kapp[cond]
kapp0 = kapp0[kapp0.isnull() == False]

In [60]:
# Get kapp
kapp0 = df_kapp[cond]
kapp0 = kapp0[kapp0.isnull() == False]
med = round(np.median(kapp0), 1)

# Map kapp to ENZLOAD entries
idx = df_enz.index.to_list()
df_write = pd.DataFrame(index=idx, columns=['id', 'kapp (1/s)', 'source'])
df_write['id'] = df_write.index.to_list()
idx_kapp = [i for i in kapp0.index if i not in rxns_excl]
kapp0 = kapp0[idx_kapp]

for i in df_enz.index:
    _,rxn,rdir,enz = extract_details_from_rxnid(i)
    if enz in ['SPONT', 'UNKNOWN']:
        df_write.loc[i, 'source'] = enz
        continue
    
    entry = rxn + '_' + rdir
    if entry in kapp0.index:
        df_write.loc[i, 'kapp (1/s)'] = kapp0[entry]
        df_write.loc[i, 'source'] = 'parameterization'
    else:
        df_write.loc[i, 'kapp (1/s)'] = med
        df_write.loc[i, 'source'] = 'kapp_median'
        
# Write to GAMS txt file
idx = df_stoich[df_stoich.coupling_type == 'rxn_enz'].index
kapp_list = []

for i in idx:
    lhs = "v('ENZLOAD-" + df_stoich.id[i][4:] + "') * " + "kapp('" + i + "')"
    kapp_list.append("'" + i + "' " + str(round(df_write.loc[i, 'kapp (1/s)'] * 3600, 6)))
    
kapp_list = ['/'] + kapp_list + ['/']
with open(outfile, 'w') as f:
    f.write('\n'.join(kapp_list))